In [1]:
!pip install pyspark


https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/from_to_dbms.html
curl -O https://repo1.maven.org/maven2/org/xerial/sqlite-jdbc/3.34.0/sqlite-jdbc-3.34.0.jar

In [1]:
import os

from pyspark.sql import SparkSession,Row
import sqlite3

In [3]:


con = sqlite3.connect('example.db')
cur = con.cursor()
# Create table
cur.execute(
    '''CREATE TABLE IF NOT EXISTS stocks
       (date text, trans text, symbol text, qty real, price real)''')
# Insert a row of data
cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")
# Save (commit) the changes
con.commit()
con.close()



In [2]:
spark = (SparkSession.builder
    .master("local")
    .appName("SQLite JDBC")
    .config(
        "spark.jars",
        "/Users/derib/data_architecture/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config(
        "spark.driver.extraClassPath",
        "/Users/derib/data_architecture/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .getOrCreate())
spark

25/04/07 23:36:51 WARN Utils: Your hostname, MacBook-Pro-de-de.local resolves to a loopback address: 127.0.0.1; using 192.168.1.17 instead (on interface en0)
25/04/07 23:36:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/04/07 23:36:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
df = spark.read.format('jdbc') \
          .options(driver='org.sqlite.JDBC', dbtable='stocks',
                   url='jdbc:sqlite:/Users/derib/data_architecture/tp_spark/example.db').load()

In [6]:
df.show()

+----------+-----+------+-----+-----+
|      date|trans|symbol|  qty|price|
+----------+-----+------+-----+-----+
|2006-01-05|  BUY|  RHAT|100.0|35.14|
+----------+-----+------+-----+-----+



In [7]:
df_other = spark.read.jdbc(url='jdbc:sqlite:/Users/derib/data_architecture/tp_spark/example.db',table="stocks")

In [8]:


# Create dummy data
data = [
    Row(date="2023-01-01", trans="BUY", symbol="AAPL", qty=50, price=150.0),
    Row(date="2023-01-02", trans="SELL", symbol="GOOG", qty=30, price=2800.0),
    Row(date="2023-01-03", trans="BUY", symbol="MSFT", qty=20, price=300.0),
    Row(date="2023-01-04", trans="SELL", symbol="AMZN", qty=10, price=3500.0)
]

# Create Spark DataFrame
dummy_df = spark.createDataFrame(data)

# Show the DataFrame
dummy_df.show()

+----------+-----+------+---+------+
|      date|trans|symbol|qty| price|
+----------+-----+------+---+------+
|2023-01-01|  BUY|  AAPL| 50| 150.0|
|2023-01-02| SELL|  GOOG| 30|2800.0|
|2023-01-03|  BUY|  MSFT| 20| 300.0|
|2023-01-04| SELL|  AMZN| 10|3500.0|
+----------+-----+------+---+------+



In [9]:
# Write the DataFrame to SQLite
dummy_df.write \
    .format('jdbc') \
    .options(driver='org.sqlite.JDBC', dbtable='stocks',
             url='jdbc:sqlite:/Users/derib/data_architecture/tp_spark/example.db') \
    .mode('append') \
    .save()

25/04/07 23:31:16 WARN JdbcUtils: Requested isolation level 1 is not supported; falling back to default isolation level 8


In [3]:
#read the data from SQlite from the gender pay gap db
df_country = spark.read.jdbc(url='jdbc:sqlite:/Users/derib/data_architecture/tp2/education_career_success.db',table="Country")
df_uni = spark.read.jdbc(url='jdbc:sqlite:/Users/derib/data_architecture/tp2/education_career_success.db',table="University")

In [4]:
df_country.show()

+---------+------------+
|countryId| countryName|
+---------+------------+
|        1|       Egypt|
|        2|     Eritrea|
|        3|South-Africa|
|        4|       China|
|        5|       India|
|        6|   Indonesia|
|        7|        Iran|
|        8|      Israel|
|        9|       Japan|
|       10|     Lebanon|
|       11|    Malaysia|
|       12|       Korea|
|       13| Philippines|
|       14|Saudi-Arabia|
|       15|      Taiwan|
|       16|    Thailand|
|       17|      Turkey|
|       18|     Vietnam|
|       19|     Austria|
|       20|     Belgium|
+---------+------------+
only showing top 20 rows



In [5]:
#Create a temporary view
df_country.createOrReplaceTempView("country")

In [16]:
spark.sql("SELECT * FROM country WHERE countryName='China'").show()

+---------+-----------+
|countryId|countryName|
+---------+-----------+
|        4|      China|
+---------+-----------+



In [6]:
df_uni.createOrReplaceTempView("university")
spark.sql("SELECT * FROM university WHERE universityName='Tsinghua University'").show()

+------------+-----------------+-------------------+---------+
|universityId|universityRanking|     universityName|countryId|
+------------+-----------------+-------------------+---------+
|        1310|               44|Tsinghua University|        4|
+------------+-----------------+-------------------+---------+



In [13]:
spark.sql("""
    SELECT 
        uni.universityName, 
        uni.universityRanking, 
        c.countryName
    FROM university uni
    INNER JOIN country c 
    ON uni.countryId = c.countryId
""").show()

+--------------------+-----------------+-----------+
|      universityName|universityRanking|countryName|
+--------------------+-----------------+-----------+
|      Háskóli slands|              739|    Iceland|
|Edith Cowan Unive...|              746|  Australia|
|  Murdoch University|              605|  Australia|
|   Curtin University|              309|  Australia|
|The University of...|              237|  Australia|
| Victoria University|              863|  Australia|
|Swinburne Univers...|              452|  Australia|
| La Trobe University|              387|  Australia|
|   Deakin University|              293|  Australia|
|     RMIT University|              247|  Australia|
|The University of...|               68|  Australia|
|   Monash University|               57|  Australia|
| Flinders University|              532|  Australia|
|University of Sou...|              399|  Australia|
|The University of...|              188|  Australia|
|Central Queenslan...|              996|  Aust

In [19]:
#SELECT the 20 first unviersities in the world

In [ ]:
#SELECT the universities in Switzerland

In [ ]:
#CREATE a temporary view of all the other tables in the database


In [ ]:
#Use join to get all the data from all the tables